In [3]:
import pandas as pd
from docx import Document
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from datetime import datetime
import json


def replace_bookmark_pair(doc, pair):
    bookmark_name, replacement = pair
    found = False

    def replace_in_element(element):
        nonlocal found
        for child in element:
            if child.tag.endswith('bookmarkStart') and child.get(qn('w:name')) == bookmark_name:
                found = True
                next_sibling = child.getnext()
                while next_sibling is not None:
                    if next_sibling.tag.endswith('r'):
                        text_element = next_sibling.find('.//{http://schemas.openxmlformats.org/wordprocessingml/2006/main}t')
                        if text_element is not None:
                            text_element.text = str(replacement)
                            following_sibling = next_sibling.getnext()
                            while following_sibling is not None and not following_sibling.tag.endswith('bookmarkEnd'):
                                parent = following_sibling.getparent()
                                parent.remove(following_sibling)
                                following_sibling = next_sibling.getnext()
                            return
                    next_sibling = next_sibling.getnext()
            replace_in_element(child)  # Llamada recursiva

    replace_in_element(doc._element)

    if not found:
        print(f"Marcador '{bookmark_name}' no encontrado")


def obtenerRespuestas(dataframe, mapa_respuestas):
    """
    Genera un diccionario con el conteo de respuestas para cada pregunta del DataFrame utilizando value_counts.
    
    Args:
        dataframe (pd.DataFrame): DataFrame donde las columnas son las preguntas y los valores las respuestas.
        mapa_respuestas (dict): Diccionario con las respuestas posibles y sus valores numéricos.
    
    Returns:
        dict: Diccionario con las claves en formato 'PREGUNTA_X_Y' y los valores como conteo.
    """
    conteo_respuestas = {}
    respuestas_posibles = list(mapa_respuestas.keys())
    valores_posibles = list(mapa_respuestas.values())
    
    for i, pregunta in enumerate(dataframe.columns, start=1):
        # Contar respuestas para la pregunta
        conteo = dataframe[pregunta].value_counts()
        # Rellenar valores faltantes para las respuestas posibles
        for respuesta, valor in zip(respuestas_posibles, valores_posibles):
            clave = f"PREGUNTA_{i}_{valor}"
            conteo_respuestas[clave] = conteo.get(respuesta, 0)  # Obtener el conteo o 0 si no aparece
    
    return conteo_respuestas

def calcularValores(respuestas_agrupadas):
    reemplazos = {
        "MEDIA_INTRINSECA": round(respuestas_agrupadas['Satisfaccion_Intrinseca'].mean(), 2),
        "MEDIA_EXTRINSECA": round(respuestas_agrupadas['Satisfaccion_Extrinseca'].mean(), 2),
        "MEDIA_GENERAL": round(respuestas_agrupadas['Satisfaccion_General'].mean(), 2),
        "STD_INTRINSECA": round(respuestas_agrupadas['Satisfaccion_Intrinseca'].std(), 2),
        "STD_EXTRINSECA": round(respuestas_agrupadas['Satisfaccion_Extrinseca'].std(), 2),
        "STD_GENERAL": round(respuestas_agrupadas['Satisfaccion_General'].std(), 2),
    }

    return reemplazos

def generarWord(plantilla_doc, reemplazos):
    doc = Document(plantilla_doc)

    # Aplicar reemplazos usando map
    list(map(lambda pair: replace_bookmark_pair(doc, pair), reemplazos.items()))

    output_doc = "Informe_Satisfaccion_Generado.docx"
    doc.save(output_doc)

In [4]:
empresa = input("Por favor, indica el nombre de la empresa:")
print("Nombre de la empresa: " + empresa)

informacion = {
    "FECHA": datetime.today().strftime('%d-%m-%Y'),
    "NOMBRE_EMPRESA": empresa,
    "NOMBRE_EMPRESA2": empresa,
}

with open("informacion_prl.json", "r", encoding="utf-8") as f:
    valores_defecto = json.load(f)

# Añadir al diccionario 'informacion'
informacion.update(valores_defecto)

'''
valores_defecto = {}
with open("informacion_prl.txt", "r", encoding="utf-8") as f:
    for line in f:
        if "=" in line:
            clave, valor = line.strip().split("=", 1)
            valores_defecto[clave] = valor

informacion.update(valores_defecto)
'''


respuestas = pd.read_csv('Encuesta_ficticia.csv', sep=';')

# Obtener las preguntas directamente de las cabeceras del CSV
preguntas = list(respuestas.columns)

# Separar las preguntas en intrínsecas (pares) y extrínsecas (impares)
preguntas_intrinsecas = [q for i, q in enumerate(preguntas) if (i + 1) % 2 == 0]
preguntas_extrinsecas = [q for i, q in enumerate(preguntas) if (i + 1) % 2 != 0]

# Mapear respuestas textuales a valores numéricos
mapa_respuestas = {
    "Muy insatisfecho": 1,
    "Insatisfecho": 2,
    "Moderadamente insatisfecho": 3,
    "Ni satisfecho ni insatisfecho": 4,
    "Moderadamente satisfecho": 5,
    "Satisfecho": 6,
    "Muy satisfecho": 7
}

# Convertir respuestas textuales a numéricas usando el mapeo
respuestas_convertidas = respuestas.copy()
respuestas_convertidas = respuestas_convertidas.map(lambda x: mapa_respuestas.get(x, x) if x in mapa_respuestas else x)

# Calcular las puntuaciones
respuestas_agrupadas = pd.DataFrame()
respuestas_agrupadas['Satisfaccion_Intrinseca'] = respuestas_convertidas[preguntas_intrinsecas].sum(axis=1)
respuestas_agrupadas['Satisfaccion_Extrinseca'] = respuestas_convertidas[preguntas_extrinsecas].sum(axis=1)
respuestas_agrupadas['Satisfaccion_General'] = respuestas_agrupadas['Satisfaccion_Intrinseca'] + respuestas_agrupadas['Satisfaccion_Extrinseca']


calculos = calcularValores(respuestas_agrupadas)

conteo_respuestas = obtenerRespuestas(respuestas, mapa_respuestas)

resultados = informacion | calculos | conteo_respuestas

# Cargar el documento Word plantilla
plantilla_doc = "plantilla_informe_satisfaccion_laboral.docx"  # TODO Cambiar nombre de la plantilla

generarWord(plantilla_doc, resultados)

Nombre de la empresa: hm
Marcador 'SERVICIO_PRL' no encontrado
Marcador 'RESPONSABLE' no encontrado
Marcador 'CARGO' no encontrado
Marcador 'PREGUNTA_5_1' no encontrado
Marcador 'PREGUNTA_5_2' no encontrado
Marcador 'PREGUNTA_5_3' no encontrado
Marcador 'PREGUNTA_5_4' no encontrado
Marcador 'PREGUNTA_5_5' no encontrado
Marcador 'PREGUNTA_5_6' no encontrado
Marcador 'PREGUNTA_5_7' no encontrado
Marcador 'PREGUNTA_6_1' no encontrado
Marcador 'PREGUNTA_6_2' no encontrado
Marcador 'PREGUNTA_6_3' no encontrado
Marcador 'PREGUNTA_6_4' no encontrado
Marcador 'PREGUNTA_6_5' no encontrado
Marcador 'PREGUNTA_6_6' no encontrado
Marcador 'PREGUNTA_6_7' no encontrado
Marcador 'PREGUNTA_7_1' no encontrado
Marcador 'PREGUNTA_7_2' no encontrado
Marcador 'PREGUNTA_7_3' no encontrado
Marcador 'PREGUNTA_7_4' no encontrado
Marcador 'PREGUNTA_7_5' no encontrado
Marcador 'PREGUNTA_7_6' no encontrado
Marcador 'PREGUNTA_7_7' no encontrado
Marcador 'PREGUNTA_8_1' no encontrado
Marcador 'PREGUNTA_8_2' no encont

PermissionError: [Errno 13] Permission denied: 'Informe_Satisfaccion_Generado.docx'